In [1]:
import setup
setup.init_django()
from decouple import config
from openai import OpenAI

EMBEDDING_MODEL = config("EMBEDDING_MODEL", default="text-embedding-3-small")
EMBEDDING_DIMENSIONS = 1536
RECREATE_DATA = True

In [2]:
sentences = [
    "The weather is lovely today.",
    "It's so sunny outside!",
    "He drove to the stadium.",
]

In [3]:
from blog.models import Post

new_data = []
for i,x in enumerate(sentences):
    new_data.append(
        Post(
            title=f"Blog Post #{i+1}",
            content=x,
            can_delete=True
        )
    )

if RECREATE_DATA:
    qs = Post.objects.filter(can_delete=True)
    qs.delete()
    Post.objects.bulk_create(new_data)

In [4]:
qs = Post.objects.filter(can_delete=True)
qs.count()

3

In [5]:
client = OpenAI(
    api_key=config("OPENAI_API_KEY")
)

In [6]:
def get_embedding(text, model=EMBEDDING_MODEL):
    text = text.replace("\n", " ").strip()
    return client.embeddings.create(input=[text], model=model).data[0].embedding

In [7]:
for obj in qs:
    if obj.embedding is None:
        obj.embedding = get_embedding(obj.get_embedding_text_raw())
        obj.save()

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}